# The fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


# Wcode Class

In [41]:
class Wcode < Hash
  def initialize(word = Group::Identity)
    @words = [word]
    #---
    Rseq.each{|x| self.store(x.char, [])}
    if word != Group::Identity
      word.flatten.factors.each_with_index do |c, i|
        k = i
        self[c.char].insert(-1, 2*k)
        self[c.inverse.char].insert(0,2*k+1)
      end
    end
  end
  attr_reader :words
  #---
  def show()
    self.values.map(&:to_s).join
  end
  def to_a()
    self.values.flatten
  end
  def size()
    self.to_a.size
  end
  def copy()
    wc = self.class.new
    wc.each_key{|key| wc[key] = self[key].dup}
    wc
  end
  #---
  def narrow()
    wc = self.copy
    sorted = wc.values.flatten.sort
    wc.each_key do |key|
      wc[key].map!{|v| sorted.index(v)}
    end
    wc
  end
  #---
  def to_word()
    chars = (0..self.size/2).to_a.map{|i| self.map{|key, arr| key if arr.include?(2*i)}.join}
    Word.new(chars.join)
  end
  #---
  def product(another)
    wc = self.copy
    size = self.size
    wc.map do |k, v|
      another[k].each do |n|
        n = n + self.size
        (n.odd?) ? v = [n]+v : v << n 
      end
      wc[k] = v
    end
    wc
  end
  #---
  def devide(odd_nums)
    facs = []
    facs[0] = self.map{|key, arr| [key, arr.select{|x| x <= odd_nums[0] or odd_nums[1] < x}] }.to_h
    facs[1] = self.map{|key, arr| [key, arr.select{|x| odd_nums[0] < x and x <= odd_nums[1]}] }.to_h
    facs.map! do |f|
      wc = self.class.new
      wc.each_key{|key| wc[key] = f[key]}
    end
    return facs
  end
  #---
  ## Turaev cobracket for Wcodes
  def cobracket(verbose=false)
    odd_pairs=(1..self.size/2).to_a.map{|x| 2*x-1}.combination(2)
    factors = odd_pairs.map do |odds|
      divs = odds.map{|odd| Division.new(self, odd)}
      linking = divs[0].linking(divs[1])
      factors = self.devide(odds).map(&:narrow)
      #---
      {odds: odds, linking: linking, factors: factors }
    end
    return verbose ? factors : factors.select{|prd| prd[:linking] != 0}
  end  
end; nil

#-----------------------------------
class Division
  def initialize(wc, odd)
    @wc = wc
    @odd = odd
    wcvf = wc.values.flatten
    @indices = [wcvf.index(@odd), wcvf.index((@odd+1)%(@wc).size)]
    ids = @indices.sort
    @interior = wcvf[ids[0]+1..ids[1]-1]
    @sign = (@indices == @indices.sort) ? 1 : -1
  end
  attr_reader :odd, :indices, :interior, :sign
  
  def linking(another)
    sign1 = @sign
    sign2 = if @interior.include?(another.odd)
              (@interior.include?(another.odd+1)) ? 0 : -1
            else
              (@interior.include?(another.odd+1)) ? 1 : 0
            end
    return sign1*sign2
  end
end;nil

In [42]:
w = comms[1] #Word.new(a[1], a[2], b[1].inverse)
wc = Wcode.new(w)
p w.show, wc.show
printf("===\n")

wcf = wc.values.flatten

divs = (1..wc.size/2).map{|n| 2*n-1}.map{|odd| Division.new(wc, odd)}

divs.each do |div|
  p div.odd, div.interior, div.sign
  printf "---\n"
end

divs.combination(2).each do |comb|
  p "#{comb.map(&:odd)} -- linking --> #{comb[0].linking(comb[1])}"
end

nil

"abAB"
"[5, 0][3, 6][1, 4][7, 2][][][][][][][][]"
===
1
[4, 7]
1
---
3
[6, 1]
1
---
5
[0, 3]
1
---
7
[3, 6, 1, 4]
-1
---
"[1, 3] -- linking --> 1"
"[1, 5] -- linking --> 0"
"[1, 7] -- linking --> -1"
"[3, 5] -- linking --> 1"
"[3, 7] -- linking --> 0"
"[5, 7] -- linking --> 0"


In [46]:
wc.cobracket(1).each do |t|
  t.each{ |k,v| p "#{k}: #{(k == :factors) ? v.map(&:show) : v}" }
  printf "---\n"
end; nil

"odds: [1, 3]"
"linking: 1"
"factors: [\"[3, 0][4][1, 2][5][][][][][][][][]\", \"[][1][][0][][][][][][][][]\"]"
---
"odds: [1, 5]"
"linking: 0"
"factors: [\"[0][2][1][3][][][][][][][][]\", \"[3][1][2][0][][][][][][][][]\"]"
---
"odds: [1, 7]"
"linking: -1"
"factors: [\"[0][][1][][][][][][][][][]\", \"[3][1, 4][2][5, 0][][][][][][][][]\"]"
---
"odds: [3, 5]"
"linking: 1"
"factors: [\"[0][3, 4][1][5, 2][][][][][][][][]\", \"[1][][0][][][][][][][][][]\"]"
---
"odds: [3, 7]"
"linking: 0"
"factors: [\"[0][3][1][2][][][][][][][][]\", \"[1][2][0][3][][][][][][][][]\"]"
---
"odds: [5, 7]"
"linking: 0"
"factors: [\"[5, 0][3][1, 4][2][][][][][][][][]\", \"[][0][][1][][][][][][][][]\"]"
---


## Turaev cobraket

In [47]:
def cobracket(myw, verbose=false)  
  monomials = Wcode.new(myw).cobracket(verbose).map do |mono|
    prewords = mono[:factors].map(&:narrow).map(&:to_word)
    words = prewords.map(&:cyclic_reduce)
    mono.merge({coeff: mono[:linking], words: words})
  end
  
  result = []
  while monomials.size > 0
    pop = monomials.pop
    #---
    monomials.each do |mono|
      if mono[:words][0].conjugate?(pop[:words][0]) and mono[:words][1].conjugate?(pop[:words][1])
        pop[:coeff] += mono[:coeff]
        pop[:order] += mono[:order]
        mono[:coeff] = nil
      elsif mono[:words][0].conjugate?(pop[:words][1]) and mono[:words][1].conjugate?(pop[:words][0])
        pop[:coeff] += mono[:coeff]*(-1)
        pop[:order] += mono[:order]
        mono[:coeff] = nil
      end
    end
    #---
    if pop[:coeff] != 0 or verbose
#      result << ((pop[:coeff] == 1) ? '' : "(#{pop[:coeff].to_s}) ") + pop[:words].join(" \u{2227} ")
      result << pop
    end
    #---
    monomials.delete_if{|mono| mono[:coeff].nil?}
  end 
  return (result.empty?) ? [{orders: [], linking: 0, factors: [], coeff: 0, words: []}] : result
end; nil

# Execution

In [48]:
a1, a2 = Word.new(a[1]), Word.new(a[2])
c = (a1*Word.new(b[1])*(a1.inverse)).flatten

samples = [
  a1, 
  a1*Word.new(b[2]),
  a1*(Word.new(b[2]).inverse),
  #---
  a1^4,
  c,
  comms[1],
  #---
  a1*(comms[2].inverse),
  a1*comms[2],
  a1*(comms[2]^3),
  a1*(a2^3),
  a1*(Word.new(a[2], b[2].inverse)^3),
  a1*comms[2]*c*comms[2]*(c.inverse),
  #---
  a1*Group.commutator(a[2], a[3]),
  a1*(comms[2]*comms[3])*Group.commutator(a[2], a[3])*((comms[2]*comms[3]).inverse),
  #---
  a1*Group.commutator(b[1], a[2])
  ]

samples.each do |s|
  ws = s.show.gsub(/[()]+/, '.').gsub(/^[.]|[.]$/, '')
  cob = cobracket(s).map do |h|
    coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
    coeff + "#{h[:words].join(" \u{2227} ")}"
  end
  #---
  p "#{ws}  |--\u{03B4}-->  #{cob.join(' + ')}"
end; nil

"a  |--δ-->  (0)"
"a.d  |--δ-->  (-1)a ∧ d"
"a.D  |--δ-->  (0)"
"a.a.a.a  |--δ-->  aaa ∧ a + aa ∧ aa + a ∧ aaa"
"abA  |--δ-->  1 ∧ b"


NoMethodError: undefined method `+' for nil:NilClass

"a  |--δ-->  0"
"a.d  |--δ-->  a ∧ d"
"a.D  |--δ-->  0"
"a.a.a.a  |--δ-->  (-1) aa ∧ aa"
"abA  |--δ-->  (-1) 1 ∧ b"
"abAB  |--δ-->  0"
"a.dcDC  |--δ-->  a ∧ dcDC"
"a.cdCD  |--δ-->  0"
"a.cdCD.cdCD.cdCD  |--δ-->  (-1) acdCDcdCD ∧ cdCD + (-1) acdCD ∧ cdCDcdCD"
"a.c.c.c  |--δ-->  (-1) acc ∧ c + (-1) ac ∧ cc"
"a.cD.cD.cD  |--δ-->  (-1) acDcD ∧ cD + (-1) acD ∧ cDcD"
"a.cdCD.abA.cdCD.aBA  |--δ-->  (-1) acdCD ∧ cdCD + (-1) cdCD ∧ cdCDa"
"a.ceCE  |--δ-->  aceC ∧ E + (-1) ace ∧ CE + ac ∧ C"
"a.cdCD.efEF.ceCE.feFEdcDC  |--δ-->  acdCDefEFceCfeFEdcDC ∧ E + (-1) acdCDefEFcefeFEdcDC ∧ CE + acdCDefEFcfeFEdcDC ∧ C"

### TODO
$[a_{1},b_{1}]$ の処理において、
* [x] [5, 0, 6, 7] の sign が正しくないようだ。
* [x] a(x)A [ $a_{1}\otimes a_{1}^{-1}$ ] の集計もおかしい。
* [x] $a_{1}^{3}$ の計算がエラーになる。
* [ ] simple subword たちによる計算を実装する。